<a href="https://colab.research.google.com/github/udayanand22/Lanchain/blob/main/narada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain langchain-cohere cohere transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install langchain_community


In [4]:
import cohere
from langchain_cohere import ChatCohere
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableSequence

# Initialize Cohere
cohere_api_key = "nWna0uV7o3mC4IsL9WFejBV4wOf8BgB1YDB0Ew8D"  # Replace with your key
llm = ChatCohere(cohere_api_key=cohere_api_key, model="command-r")

# Hugging Face Summarizer and QA
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# Setup memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Prompt with memory placeholder
prompt = PromptTemplate.from_template(
    """You are Narada, the celestial sage, known for your dramatic flair and poetic storytelling.
{chat_history}
User: Tell me a story about {topic}
Narada:"""
)

# Chain with memory
story_chain = LLMChain(prompt=prompt, llm=llm, memory=memory)

print("🦚 Narada Cohere Edition Activated — Now With Memory!")

while True:
    topic = input("🎤 Ask Narada for a story (or type 'exit'): ")
    if topic.lower() == 'exit':
        print("Goodbye! Narada's tales will await you next time.")
        break

    # Generate raw story with memory context
    raw_story = story_chain.run({"topic": topic})

    if raw_story.strip() == "":
        print("\n⚠️ Narada was unable to weave a tale. Please try another topic.")
        continue

    print("\n📜 Raw Narada Tale:\n", raw_story)

    try:
        summary = summarizer(raw_story, max_length=100, min_length=30, do_sample=False)[0]['summary_text']
        print("\n🪶 Narada's Summary:\n", summary)

        # 👇 Memory print here
        print("\n🧠 Narada's Memory So Far:\n")
        for msg in memory.chat_memory.messages:
            print(f"{msg.type.title()}: {msg.content}")

        # QA relevance check
        result = qa_pipeline(question=topic, context=summary)
        score = result['score']

        if score < 0.5:
            print("\n⚠️ The answer seems to not align well with the question. Modifying the response...\n")
            modified_summary = f"Here's a more detailed answer about {topic}: {raw_story}"
            print("\n🪶 Modified Narada's Summary:\n", modified_summary)
        else:
            print("\n✅ Narada's Summary is aligned with the question.")

    except Exception as e:
        print(f"\n⚠️ An error occurred while summarizing or comparing: {e}")


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [36]:
!pip install huggingface_hub[hf_xet]


In [37]:
# Import the required libraries
from googlesearch import search
from newspaper import Article
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


In [38]:
# Initialize the FlanT5 model for text generation and BART for summarization
generator = pipeline("text-generation", model="google/flan-t5-xl", max_length=400)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Wrap the text generation pipeline with LangChain
generator_llm = HuggingFacePipeline(pipeline=generator)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPT

In [39]:
# Define a prompt template for generating mythological tales
story_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Compose a mythological tale about {topic} in Narada's poetic and dramatic storytelling style."
)

# Create a LangChain LLMChain with the FlanT5 model
story_chain = LLMChain(llm=generator_llm, prompt=story_prompt)


In [40]:
# Web scraping function to get content from a given URL
def scrape_article(url):
    article = Article(url)  # Initialize the Article object with the URL
    article.download()  # Download the article content
    article.parse()  # Parse the content to extract text
    return article.text  # Return the article's text


In [41]:
# Function to search Google for a topic and return the first result URL
def search_topic(topic):
    query = f"{topic} site:en.wikipedia.org"  # Customize search to target Wikipedia
    search_results = list(search(query, num_results=5))  # Convert the generator to a list
    return search_results[0]  # Return the first result URL


In [42]:
# Function to generate and summarize a story based on a given topic
def generate_and_summarize_story(topic):
    try:
        # Search for the topic and get the URL of the top result
        url = search_topic(topic)

        # Scrape article content from the provided URL
        scraped_text = scrape_article(url)

        # Generate the story using LangChain and FlanT5 model (Use invoke instead of run)
        raw_story = story_chain.invoke(scraped_text)  # Updated method here

        # Summarize the story using BART summarizer
        summary = summarizer(raw_story, max_length=100, min_length=40, do_sample=False, truncation=True)[0]['summary_text']  # Truncation enabled

        return raw_story, summary
    except Exception as e:
        return f"Error: {str(e)}"  # Handle any errors gracefully


In [47]:
# Function to generate and summarize a story based on a given topic
def generate_and_summarize_story(topic):
    try:
        # Search for the topic and get the URL of the top result
        url = search_topic(topic)

        # Scrape article content from the provided URL
        scraped_text = scrape_article(url)

        # Generate the story using LangChain and FlanT5 model (Use invoke instead of run)
        raw_story = story_chain.invoke(scraped_text)  # Updated method here

        # Summarize the story using BART summarizer
        summary = summarizer(raw_story, max_length=100, min_length=40, do_sample=False, truncation=True)[0]['summary_text']  # Truncation enabled

        return raw_story, summary
    except Exception as e:
        # Handle any errors gracefully and return None for both values to avoid unpacking error
        print(f"Error: {str(e)}")  # Print the error for debugging
        return None, None

In [48]:
# Main loop for interacting with Narada, generating stories and summaries
def main():
    print("🦚 Narada Lite Activated — Powered by FlanT5 & BART\n")

    while True:
        try:
            # Ask the user for a topic
            topic = input("\n🎤 Ask Narada for a story (or type 'exit'): ")
            if topic.lower() in ['exit', 'quit']:
                break

            # Generate the story and summary for the searched topic
            raw_story, summary = generate_and_summarize_story(topic)

            # Display the generated story and the summary
            print("\n📜 Raw Narada Tale:\n", raw_story)
            print("\n🪶 Narada's Summary:\n", summary)

        except KeyboardInterrupt:
            print("\n👋 Exiting gracefully...")
            break

if __name__ == "__main__":
    main()


🦚 Narada Lite Activated — Powered by FlanT5 & BART


🎤 Ask Narada for a story (or type 'exit'): shiva
Error:  `args[0]`: {'topic': 'Major deity in Hinduism\n\nShiva ( ; Sanskrit: शिव, lit. \'The Auspicious One\', IAST: Śiva [ɕɪʋɐ]), also known as Mahadeva ( ; Sanskrit: महादेव:, lit. \'The Great God\', IAST: Mahādevaḥ, [mɐɦaːd̪eːʋɐh]) and Hara, is one of the principal deities of Hinduism.[19] He is the Supreme Being in Shaivism, one of the major traditions within Hinduism.\n\nShiva is known as The Destroyer within the Trimurti, the Hindu trinity which also includes Brahma and Vishnu. In the Shaivite tradition, Shiva is the Supreme Lord who creates, protects and transforms the universe. In the goddess-oriented Shakta tradition, the Supreme Goddess (Devi) is regarded as the energy and creative power (Shakti) and the equal complementary partner of Shiva. Shiva is one of the five equivalent deities in Panchayatana puja of the Smarta tradition of Hinduism.\n\nShiva has many aspects, benevole

In [51]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import json

# Load Flan-T5 (or switch to "facebook/bart-large")
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def generate_story(topic_text):
    prompt = f"Tell a short story about: {topic_text}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = model.generate(**inputs, max_length=256, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def narada_lite_terminal():
    print("🦚 Narada Lite Activated — Powered by Flan-T5\n")
    while True:
        user_input = input("🎤 Ask Narada for a story (or type 'exit'): ").strip()
        if user_input.lower() == 'exit':
            print("🕉️ Narada Lite Exiting. May wisdom guide you.")
            break

        try:
            with open("topic_context.json", "r") as f:
                data = json.load(f)  # ✅ Correctly parse JSON
            topic_data = data["args"][0]["topic"]  # ✅ Access like a dict

            story = generate_story(topic_data)
            print("\n📖 Narada's Tale:\n", story, "\n")

        except Exception as e:
            print("⚠️ Error:", str(e))
            print("Please ensure 'topic_context.json' is correctly formatted.\n")

# Optional: run directly
# narada_lite_terminal()


In [52]:
# Main loop for interacting with Narada, generating stories and summaries
def main():
    print("🦚 Narada Lite Activated — Powered by FlanT5 & BART\n")

    while True:
        try:
            # Ask the user for a topic
            topic = input("\n🎤 Ask Narada for a story (or type 'exit'): ")
            if topic.lower() in ['exit', 'quit']:
                break

            # Generate the story and summary for the searched topic
            raw_story, summary = generate_and_summarize_story(topic)

            # Display the generated story and the summary
            print("\n📜 Raw Narada Tale:\n", raw_story)
            print("\n🪶 Narada's Summary:\n", summary)

        except KeyboardInterrupt:
            print("\n👋 Exiting gracefully...")
            break

if __name__ == "__main__":
    main()


🦚 Narada Lite Activated — Powered by FlanT5 & BART


🎤 Ask Narada for a story (or type 'exit'): shiv
Error:  `args[0]`: {'topic': 'Improvised knife-like weapon\n\nShivs hidden in a book, Hong Kong\n\nA shiv, also chiv, schiv, shivvie or shank,[1][2] is a handcrafted bladed weapon resembling a knife that is commonly associated with prison inmates.\n\nSince weapons are prohibited in prisons, the intended mode of concealment is central to a shiv\'s construction. An especially thin handle, for instance, makes it easier to conceal in available cracks or crevices in the prison\'s construction, or in stacks of objects, such as books, permitted to the prisoners; however, this can also render the shiv difficult to grip and wield. Routine body searches in prison make it difficult to conceal a shiv on one\'s person on a continuous basis. As well as the prison authorities, it is also desirable to conceal possession of a shiv from members of rival prison populations.\n\nThe word is recorded from th